### Naver
- html parsing으로 진행

In [3]:
import requests
from bs4 import BeautifulSoup

In [8]:
response = requests.get("https://comic.naver.com/webtoon/genre.nhn?genre=story")
dom = BeautifulSoup(response.content, "html.parser") # utf8 확인, 그래서 encoding은 넘어감

In [9]:
keywords = dom.select(".list_area > .img_list > li")

In [10]:
df = pd.DataFrame(columns=['unique_id','title','author','total_score','main_link','image_link','update_date'])

In [11]:
dom.select("dl > dt > a")[0].get('href')[-6:] # unique id

'641253'

In [12]:
keywords[0].select_one("dl > dt > a").text # title

'외모지상주의'

In [13]:
keywords[0].select_one("dl > dd > a").text # author

'박태준'

In [14]:
keywords[0].select_one("dl > dd > .rating_type > strong").text # total score

'9.54'

In [15]:
"https://comic.naver.com" + dom.select("dl > dt > a")[0].get('href') # main link

'https://comic.naver.com/webtoon/list.nhn?titleId=641253'

In [131]:
dom.select(".list_area > .img_list > li > .thumb > a > img")[0].get('src') # image link

'https://shared-comic.pstatic.net/thumb/webtoon/641253/thumbnail/title_thumbnail_20141120112141_t83x90.jpg'

In [132]:
keywords[0].select_one("dl > dd.date2").text # update date

'2018.08.23'

##### 각 column들의 값을 가져오는 경로가 다르므로, 일단 나눠서 진행.

In [133]:
dic = {}
dic['unique_id'] = []
dic['title'] = []
dic['author'] = []
dic['total_score'] = []
dic['main_link'] = []
dic['image_link'] = []
dic['update_date'] = []

In [134]:
# title, author, total score
for keyword in keywords:
    dic['title'].append(keyword.select_one("dl > dt > a").text)
    dic['author'].append(keyword.select_one("dl > dd > a").text)
    dic['total_score'].append(keyword.select_one("dl > dd > .rating_type > strong").text)
    dic['update_date'].append(keyword.select_one("dl > dd.date2").text)

In [135]:
# unique id 
unique_id_list = dom.select("dl > dt > a")
for i in range(len(unique_id_list)):
    idx = unique_id_list[i].get('href').index("=")
    dic['unique_id'].append(unique_id_list[i].get('href')[idx+1:])

In [136]:
# main link
main_link_list = dom.select("dl > dt > a")
for i in range(len(main_link_list)):
    dic['main_link'].append("https://comic.naver.com" + main_link_list[i].get('href'))

In [137]:
# image link
image_link_list = dom.select(".list_area > .img_list > li > .thumb > a > img:nth-of-type(1)")
for i in range(len(image_link_list)):
    dic['image_link'].append(image_link_list[i].get('src'))

In [138]:
for i in set(dic.keys()):
    print(i ,len(dic[i]))

total_score 552
update_date 552
author 552
image_link 552
unique_id 552
title 552
main_link 552


In [112]:
dic['image_link'][1]

'https://shared-comic.pstatic.net/thumb/webtoon/22052/thumbnail/thumbnail_title_22052_83x90.gif'

In [139]:
df = pd.DataFrame(dic)

In [140]:
df.tail()

,unique_id,title,author,total_score,main_link,image_link,update_date
547,622643,진눈깨비 소년,쥬드프라이데이,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.23
548,703840,한국만화 19...,웹툰작가,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.05.17
549,336945,길에서 만나다,쥬드 프라이데이,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2013.01.17
550,515883,사또(Satt...,최윤진,9.80,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2015.12.29
551,347685,슈퍼트리오 시...,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04


In [115]:
link = "https://comic.naver.com/webtoon/genre.nhn?genre=episode"
link.index("=")

47

In [116]:
link[48:]

'episode'

In [247]:
dic = {}

url = "https://comic.naver.com/webtoon/genre.nhn?view=list&order=ViewCount&genre=story"
response = requests.get(url)
dom = BeautifulSoup(response.content, "html.parser")
keywords = dom.select("#content > div.list_area.table_list_area > table > tbody > tr")
keywords[0].select_one("td > a > strong").text
    

'외모지상주의'

#### 함수화

In [8]:
# 함수화
def naver_webtoon(genre):
    
    dic = {}
    dic['unique_id'] = []
    dic['title'] = []
    dic['author'] = []
    dic['total_score'] = []
    dic['main_link'] = []
    dic['image_link'] = []
    dic['update_date'] = []
    
    # first url for title
    # get title with full name
    url = "https://comic.naver.com/webtoon/genre.nhn?view=list&order=ViewCount&genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#content > div.list_area.table_list_area > table > tbody > tr")
    for keyword in keywords:
        dic['title'].append(keyword.select_one("td > a > strong").text)
        dic['author'].append(keyword.select_one("td:nth-of-type(3) > a").text)    
    
    # second url for the others
    url =  "https://comic.naver.com/webtoon/genre.nhn?genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".list_area > .img_list > li")


    # author, total score
    for keyword in keywords:
        dic['total_score'].append(keyword.select_one("dl > dd > .rating_type > strong").text)
        dic['update_date'].append(keyword.select_one("dl > dd.date2").text)

    # unique id 
    unique_id_list = dom.select("dl > dt > a")
    for i in range(len(unique_id_list)):
        idx = unique_id_list[i].get('href').index("=")
        dic['unique_id'].append(unique_id_list[i].get('href')[idx+1:])

    # main link
    main_link_list = dom.select("dl > dt > a")
    for i in range(len(main_link_list)):
        dic['main_link'].append("https://comic.naver.com" + main_link_list[i].get('href'))

    # image link
    image_link_list = dom.select(".list_area > .img_list > li > .thumb > a > img:nth-of-type(1)")
    for i in range(len(image_link_list)):
        dic['image_link'].append(image_link_list[i].get('src'))
        
    df = pd.DataFrame(dic)
    df[genre] = genre
    df['author'] = df['author'].map(lambda x : "".join(x.split()))
    return df

In [249]:
episode = naver_webtoon("episode")
for i in set(episode.keys()):
    print(i ,len(episode[i]))

update_date 114
episode 114
image_link 114
total_score 114
title 114
author 114
main_link 114
unique_id 114


In [250]:
story = naver_webtoon("story")
for i in set(story.keys()):
    print(i ,len(story[i]))

story 552
update_date 552
image_link 552
total_score 552
title 552
author 552
main_link 552
unique_id 552


In [251]:
omnibus = naver_webtoon("omnibus")
for i in set(omnibus.keys()):
    print(i, len(omnibus[i]))

update_date 78
image_link 78
total_score 78
title 78
omnibus 78
author 78
main_link 78
unique_id 78


In [9]:
story_df = naver_webtoon("story")
story_df.tail()

,unique_id,title,author,total_score,main_link,image_link,update_date,story
548,622643,진눈깨비 소년,쥬드프라이데이,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.30,story
549,703840,한국만화 1990,웹툰작가,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.05.17,story
550,336945,길에서 만나다,쥬드프라이데이,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2013.01.17,story
551,515883,사또(Satto),최윤진,9.80,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2015.12.29,story
552,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story


In [21]:
story_df['unique_id']

0      641253
1      325629
2      570503
3      150388
4      697685
5      318995
6      662774
7      626907
8      183559
9      703846
10      26309
11     597447
12      22896
13     179704
14     648419
15      26205
16     650305
17     602910
18      67235
19      26310
20      65410
21     697656
22     568986
23     701081
24     708378
25      25455
26     434644
27     400737
28     667573
29     654774
        ...  
522    710390
523    675474
524    702165
525    675829
526    647948
527    258207
528    260510
529    310616
530    630386
531    658823
532    502673
533    670131
534    664406
535    612767
536    548870
537    700361
538    686669
539    423376
540    619329
541    517252
542    113586
543    578109
544    551648
545    669357
546    668103
547    622643
548    703840
549    336945
550    515883
551    347685
Name: unique_id, Length: 552, dtype: object

#### 장르 통합  "/"
- main genre : 에피소드, 옴니버스, 스토리
- sub genre : 일상, 개그, 판타지, 액션, 드라마, 순정, 감성, 스릴러, 시대극, 스포츠

In [10]:
genres = ["episode","omnibus","story","daily","comic","fantasy","action","drama","pure","sensibility","thrill","historical","sports"]

In [11]:
df = naver_webtoon("episode")

In [12]:
for genre in genres[1:]:
    print(genre)
    sample = naver_webtoon(genre)
    df = pd.merge(df,sample, how='outer')
df.reset_index(inplace=True, drop=True)
df.tail()

omnibus
story
daily
comic
fantasy
action
drama
pure
sensibility
thrill
historical
sports


,unique_id,title,author,total_score,main_link,image_link,update_date,episode,omnibus,story,daily,comic,fantasy,action,drama,pure,sensibility,thrill,historical,sports
740,622643,진눈깨비 소년,쥬드프라이데이,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.30,NaN,NaN,story,NaN,NaN,NaN,NaN,NaN,pure,sensibility,NaN,NaN,NaN
741,703840,한국만화 1990,웹툰작가,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.05.17,NaN,NaN,story,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,336945,길에서 만나다,쥬드프라이데이,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2013.01.17,NaN,NaN,story,NaN,NaN,NaN,NaN,drama,NaN,sensibility,NaN,NaN,NaN
743,515883,사또(Satto),최윤진,9.80,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2015.12.29,NaN,NaN,story,NaN,NaN,fantasy,NaN,drama,NaN,NaN,NaN,NaN,NaN
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,NaN,NaN,story,NaN,comic,NaN,NaN,drama,NaN,NaN,NaN,NaN,NaN


In [6]:
col_genres = df[genres]
col_genres.fillna("", inplace=True)
col_genres

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,episode,omnibus,story,daily,comic,fantasy,action,drama,pure,sensibility,thrill,historical,sports
0,episode,,,daily,comic,,,,,,,,
1,episode,,,daily,comic,,,,,,,,
2,episode,,,,comic,,,,,,,,
3,episode,,,,comic,,,,,,,,
4,episode,,,daily,comic,,,,,,,,
5,episode,,,daily,comic,,,,,,,,
6,episode,,,daily,comic,,,,,,,,
7,episode,,,daily,comic,,,,,,,,
8,episode,,,daily,,,,drama,,,thrill,,
9,episode,,,daily,comic,,,,,,,,


In [14]:
result = []
for i in range(len(col_genres)):
    ls = list(filter(lambda value: value != "", col_genres.loc[i].values))
    result.append(ls)

In [17]:
col_genres['genre'] = np.array(["/".join(ls) for ls in result])
col_genres

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,episode,omnibus,story,daily,comic,fantasy,action,drama,pure,sensibility,thrill,historical,sports,genre
0,episode,,,daily,comic,,,,,,,,,episode/daily/comic
1,episode,,,daily,comic,,,,,,,,,episode/daily/comic
2,episode,,,,comic,,,,,,,,,episode/comic
3,episode,,,,comic,,,,,,,,,episode/comic
4,episode,,,daily,comic,,,,,,,,,episode/daily/comic
5,episode,,,daily,comic,,,,,,,,,episode/daily/comic
6,episode,,,daily,comic,,,,,,,,,episode/daily/comic
7,episode,,,daily,comic,,,,,,,,,episode/daily/comic
8,episode,,,daily,,,,drama,,,thrill,,,episode/daily/drama/thrill
9,episode,,,daily,comic,,,,,,,,,episode/daily/comic


#### 함수화 - 장르통합 "/"

In [13]:
def make_genre(df):
    genres = ["episode","omnibus","story","daily","comic","fantasy","action","drama","pure","sensibility","thrill","historical","sports"]
    df.fillna("", inplace=True)

    result = []
    for i in range(len(df[genres])):
        ls = list(filter(lambda value: value != "", df[genres].loc[i].values))
        result.append(ls)

    df['genre'] = np.array(["/".join(ls) for ls in result])
    df.drop(columns=genres, axis=1, inplace=True)
    return df

In [14]:
test = df.copy()
test = make_genre(test)
test.tail()

,unique_id,title,author,total_score,main_link,image_link,update_date,genre
740,622643,진눈깨비 소년,쥬드프라이데이,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.30,story/pure/sensibility
741,703840,한국만화 1990,웹툰작가,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.05.17,story
742,336945,길에서 만나다,쥬드프라이데이,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2013.01.17,story/drama/sensibility
743,515883,사또(Satto),최윤진,9.80,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2015.12.29,story/fantasy/drama
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama


In [15]:
test.to_csv("data_Juno/webtoon_naver_info_by_genres.csv", index=False)

### Daum

In [2]:
from pandas.io.json import json_normalize
import requests

In [ ]:
def get_data(url):
    response = requests.get(url)
    json_info = response.json()